In [13]:
import requests as Requests
import json as JSON
import http.server
import socketserver
from http.server import BaseHTTPRequestHandler
from sklearn.linear_model import LinearRegression
import numpy as np
#from BaseHTTPServer import BaseHTTPRequestHandler, HTTPServer
PORT = 8005

import requests as Requests
import json as JSON
import http.server
import socketserver
import collections
from http.server import BaseHTTPRequestHandler
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import numpy as np
import datetime
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from decimal import *
#from BaseHTTPServer import BaseHTTPRequestHandler, HTTPServer


In [14]:
class OLSR():
    
    def __init__(self,Train_Windows,Train_Actuals,Test_Window):
    
        self.Windows = Train_Windows;
        self.Actuals = Train_Actuals;
        self.Test = Test_Window;
        self.Model = LinearRegression();
        self.Train()
        
        
    def Train(self):
        self.Model.fit(self.Windows,self.Actuals);
    
    def Predict(self,Data=''):
        
        if len(Data) == 0 :
            return self.Model.predict(self.Test)
        return self.Model.predict(Data)
    
    def Score(self,Data,Actuals):
        
        return self.Model.score(Data,Actuals)
    
    
    def MSE(self,Actuals):

        return mean_squared_error(Actuals,self.Predictions)


In [ ]:
class MLPR():
    
    
    def __init__(self,Train_Windows,Train_Actuals,Test_Window):
    
       # Training takes too long, train iwth only 0.5% of data set
        self.Windows = Train_Windows[int(len(Train_Windows)*0.995):len(Train_Windows)];
        self.Actuals = Train_Actuals[int(len(Train_Actuals)*0.995):len(Train_Actuals)];
        self.Test = Test_Window;
        self.Model = MLPRegressor(hidden_layer_sizes=(20,10,5),max_iter=10000000,
                                       activation='logistic');
        self.Train()
        
        self.Score
        
    def Train(self):
        self.Model.fit(self.Windows,self.Actuals);
    
    def Predict(self,Data=''):
        
        if len(Data) == 0 :
            return self.Model.predict(self.Test)
        return self.Model.predict(Data)
    
    
    def Score(self,Data,Actuals):
        
        
        return self.Model.score(Data,Actuals)
        

In [15]:
class LinearAutoRegressive():
    
    
    def __init__(self,Interval_Diff,Next_Open,Next_Start_Time,
                 Training_Windows,Close_Actuals,High_Actuals,Low_Actuals,Vol_Actuals,Test_Window,Symbol):
        
        self.CloseModel = OLSR(Training_Windows,Close_Actuals,Test_Window);
        self.LowModel = OLSR(Training_Windows,Low_Actuals,Test_Window);
        self.HighModel = OLSR(Training_Windows,High_Actuals,Test_Window);
        self.VolModel = OLSR(Training_Windows,Vol_Actuals,Test_Window);
        self.Interval_Diff = Interval_Diff;
        self.Next_Start_Time = Next_Start_Time;
        self.Next_Open_Price = Next_Open;
        self.Symbol = Symbol;
        self.Test_Window = Test_Window;
        self.Predictions = [];
        
        self.Vol_Coef = self.VolModel.Score(Training_Windows,Vol_Actuals);
        self.Low_Coef = self.LowModel.Score(Training_Windows,Low_Actuals);
        self.High_Coef = self.HighModel.Score(Training_Windows,High_Actuals);
        self.Close_Coef = self.CloseModel.Score(Training_Windows,Close_Actuals);

        
    def Predict(self,Limit=10):
        
        for i in range(Limit):
            
            # Close price of the last interval becomes the open price of the enext
            self.Next_Open_Price = self.CloseModel.Predict(self.Test_Window);
           
            # Pridict and construct a time series element
            Prediction = Candlestick_Recognition([[self.Next_Start_Time, self.LowModel.Predict(self.Test_Window)[0],
                                             self.Next_Open_Price[0],self.CloseModel.Predict(self.Test_Window)[0],
                                             self.HighModel.Predict(self.Test_Window)[0],
                                             self.VolModel.Predict(self.Test_Window)[0]]])[0];
            
            
            # Remove the first time series elemement  & append the predicted element      
            self.Test_Window = [np.concatenate([self.Test_Window[0][7:len(self.Test_Window[0])]] +[Prediction])]
        
            self.Predictions.append(Prediction)
            
            # Set start time of the next predcted time series element
            self.Next_Start_Time = int(self.Next_Start_Time + self.Interval_Diff);
      
        return self.Predictions    
            

In [ ]:
class MLPAutoRegressive():
    
    
    def __init__(self,Interval_Diff,Next_Open,Next_Start_Time,
                 Training_Windows,Close_Actuals,High_Actuals,Low_Actuals,Vol_Actuals,Test_Window,Symbol):
        
        self.CloseModel = MLPR(Training_Windows,Close_Actuals,Test_Window);
        self.LowModel = MLPR(Training_Windows,Low_Actuals,Test_Window);
        self.HighModel = MLPR(Training_Windows,High_Actuals,Test_Window);
        self.VolModel = MLPR(Training_Windows,Vol_Actuals,Test_Window);
        self.Interval_Diff = Interval_Diff;
        self.Next_Start_Time = Next_Start_Time;
        self.Next_Open_Price = Next_Open;
        self.Symbol = Symbol;
        self.Test_Window = Test_Window;
        
        
        self.Predictions = [];
        
        self.Vol_Coef = self.VolModel.Score(Training_Windows,Vol_Actuals);
        self.Low_Coef = self.LowModel.Score(Training_Windows,Low_Actuals);
        self.High_Coef = self.HighModel.Score(Training_Windows,High_Actuals);
        self.Close_Coef = self.CloseModel.Score(Training_Windows,Close_Actuals);

        
        
        
        
        
        
        
    def Predict(self,Limit=10):
        
        for i in range(Limit):
            
            # Close price of the last interval becomes the open price of the enext
            self.Next_Open_Price = self.CloseModel.Predict();
            
            # Pridict and construct a time series element
            Prediction = Candlestick_Recognition([[self.Next_Start_Time, self.LowModel.Predict(self.Test_Window)[0],
                                             self.Next_Open_Price,self.CloseModel.Predict(self.Test_Window)[0],
                                             self.HighModel.Predict(self.Test_Window)[0],
                                             self.VolModel.Predict(self.Test_Window)[0]]])[0];
            
            
            # Remove the first time series elemement  & append the predicted element      
            self.Test_Window = [np.concatenate([self.Test_Window[0][7:len(self.Test_Window[0])]] +[Prediction])]
            
            self.Predictions.append(Prediction)
            
            
            
            # Set start time of the next predcted time series element
            self.Next_Start_Time = int(self.Next_Start_Time + self.Interval_Diff);
            
        return self.Predictions
            

In [16]:
def Doji(O,C):
    
    return (O == C)
    
def Bull_Engulfing(O,C,O1,C1):
    return (((O1 > C1) and (C > O) and (C >= O1) and (C1 >= O) and ((C - O) > (O1 - C1))))

def Bear_Engulfing(O,C,O1,C1):
    return ((C1 > O1) and (O > C) and (O >= C1) and (O1 >= C) and ((O - C) > (C1 - O1)))

def Hammer(O,C,H,L):
    return (((H-L)>3*(O-C)and((C-L)/(.001+H-L)>0.6)and((O-L)/(.001+H-L)>0.6)))

def Hanging_Man(O,C,H,L):
    return (((H - L) > 4 * (O - C)) and ((C - L) / (.001 + H - L) >= 0.75) and ((O - L) / (.001 + H - L) >= .075))

def Piercing_Line(O,C,H,L,O1,C1):
    return ((C1 < O1) and (((O1 + C1) / 2) < C) and (O < C) and (O < C1) and (C < O1) and ((C - O) / (.001 + (H - L)) > 0.6))
    
def Dark_Cloud(O,C,H,L,O1,C1):
    return ((C1 > O1) and (((C1 + O1) / 2) > C) and (O > C) and (O > C1) and (C > O1) and ((O - C) / (.001 + (H - L)) > .6))

def Bull_Harami(O,C,O1,C1):
    return ((O1 > C1) and (C > O) and (C <= O1) and (C1 <= O) and ((C - O) < (O1 - C1)))

def Bear_Harami(O,C,O1,C1):
    return ((C1 > O1) and (O > C) and (O <= C1) and (O1 <= C) and ((O - C) < (C1 - O1)))

def Morning_Star(O,C,H,L,O1,C1,H1,L1,O2,C2,H2,L2):
    return ((O2>C2)and((O2-C2)/(.001+H2-L2)>.6)and(C2>O1)and(O1>C1)and((H1-L1)>(3*(C1-O1)))and(C>O)and(O>O1))

def Evening_Star(O,C,H,L,O1,C1,H1,L1,O2,C2,H2,L2):
    return ((C2 > O2) and ((C2 - O2) / (.001 + H2 - L2) > .6) and (C2 < O1) and (C1 > O1) and ((H1 - L1) > (3 * (C1 - O1))) and (O > C) and (O < O1))
    
def Bull_Kick(O,C,O1,C1):
    return (O1 > C1) and (O >= O1) and (C > O)

def Bear_Kick(O,C,O1,C1):
    return (O1 < C1) and (O <= O1) and (C <= O)

def Shooting_Star(O,C,H,L):
    return (((H - L) > 4 * (O - C)) and ((H - C) / (.001 + H - L) >= 0.75) and ((H - O) / (.001 + H - L) >= 0.75))

def Inverted_Hammer(O,C,H,L):
    return (((H - L) > 3 * (O - C)) and ((H - C) / (.001 + H - L) > 0.6) and ((H - O) / (.001 + H - L) > 0.6))

def Nearly_Doji(O,C,H,L):
    return (abs(O - C ) <= ((H - L ) * 0.1))



In [17]:
def Candlestick_Recognition(Time_Series):
    
    # for each element
    for i in range(0,len(Time_Series)):
        # Extract time series values
        Open = Time_Series[i][2]
        Close = Time_Series[i][3]
        High = Time_Series[i][4]
        Low = Time_Series[i][1]
        
        Initial_Length = len(Time_Series[i])
        
        try:
            # Values from i-1 & i-2
            Open1 = Time_Series[i-1][2]
            Close1 = Time_Series[i-1][3]
            Open2 = Time_Series[i-2][2]
            Close2 = Time_Series[i-2][3]
            High1 = Time_Series[i-1][4]
            Low1 = Time_Series[i-1][1]
            High2 = Time_Series[i-2][4]
            Low2 = Time_Series[i-2][1]
            
            if Morning_Star(Open,Close,High,Low,Open1,Close1,High1,Low1,Open2,Close2,High2,Low2):
                Time_Series[i] = Time_Series[i] + [10]
                continue
            
            if Evening_Star(Open,Close,High,Low,Open1,Close1,High1,Low1,Open2,Close2,High2,Low2):
                Time_Series[i] = Time_Series[i] + [11]
                continue
                
        except IndexError: pass
        
        
        
        try:
            # values of open and close at i-1
            Open1 = Time_Series[i-1][2]
            Close1 = Time_Series[i-1][3]
            
            if Bull_Engulfing(Open,Close,Open1,Close1):
                Time_Series[i] = Time_Series[i] + [2]
                continue
                
            if Bear_Engulfing(Open,Close,Open1,Close1):
                Time_Series[i] = Time_Series[i] + [3]
                continue
                
            if Piercing_Line(Open,Close,High,Low,Open1,Close1):
                Time_Series[i] = Time_Series[i] + [6]
                continue
                
            if Dark_Cloud(Open,Close,High,Low,Open1,Close1):
                Time_Series[i] = Time_Series[i] + [7]
                continue
                
            if Bull_Harami(Open,Close,Open1,Close1):
                Time_Series[i] = Time_Series[i] + [8]
                continue
            
            if Bear_Harami(Open,Close,Open1,Close1):
                Time_Series[i] = Time_Series[i] + [9]
                continue
            
            if Bull_Kick(Open,Close,Open1,Close1):
                Time_Series[i] = Time_Series[i] + [12]
                continue
                
            if Bear_Kick(Open,Close,Open1,Close1):
                Time_Series[i] = Time_Series[i] + [13]
                continue
                
        
        except IndexError: pass
        
        if Doji(Open,Close):
            Time_Series[i] = Time_Series[i] + [1]
            continue
            
        if Hammer(Open,Close,High,Low):
            Time_Series[i] = Time_Series[i] + [4]
            continue
            
        if Hanging_Man(Open,Close,High,Low):
            Time_Series[i] = Time_Series[i] + [5]
            continue
            
        if Shooting_Star(Open,Close,High,Low):
            Time_Series[i] = Time_Series[i] + [14]
            continue
            
        if Inverted_Hammer(Open,Close,High,Low):
            Time_Series[i] = Time_Series[i] + [15]
            continue
            
        if Nearly_Doji(Open,Close,High,Low):
            Time_Series[i] = Time_Series[i] + [16]
            continue
        
        #If no Recognized Candlestick
        if len(Time_Series[i]) == Initial_Length:
            Time_Series[i] = Time_Series[i] + [0]
            continue
    
    return Time_Series
    

In [18]:
def Extract_Vals(Actuals,Predictions):
    
    Volume_Actual = [];
    High_Actual = [];
    Low_Actual = [];
    Close_Actual = [];

    Volume_Pred = [];
    High_Pred = [];
    Low_Pred = [];
    Close_Pred = [];

    for i in range(len(Predictions)):
        

        
        Volume_Actual.append(Actuals[i][5]);
        Close_Actual.append(Actuals[i][2]);
        Low_Actual.append(Actuals[i][1]);
        High_Actual.append(Actuals[i][4]);

        Volume_Pred.append(Predictions[i][5]);
        Close_Pred.append(Predictions[i][2]);
        Low_Pred.append(Predictions[i][1]);
        High_Pred.append(Predictions[i][4]);

        
    return [Low_Actual,Close_Actual,High_Actual,Volume_Actual,Low_Pred,Close_Pred,High_Pred,Volume_Pred]
        
        

In [19]:
def Make_Window(Window,Window_Size):
    
    return np.concatenate(Window[0:Window_Size-1])


def Extract_High(Window,Window_Size):
    
    return Window[Window_Size-1][4]

def Extract_Close(Window,Window_Size):

    return Window[Window_Size-1][3]

def Extract_Low(Window,Window_Size):
    
    return Window[Window_Size-1][1]

def Extract_Vol(Window,Window_Size):
    
    return Window[Window_Size-1][5]

def Extract_Open(Window,Window_Size):
    
    return Window[Window_Size-1][2]

def Extract_Candle(Window,Window_Size):
    
    return Window[Window_Size-1][6]

def Extract_Start(Window,Window_Size):
    
    return Window[Window_Size-1][0]


    

In [20]:
def Create_Windows(Time_Series,Window_Size):
    
    i=0;
    Close_Actuals = [];
    High_Actuals = [];
    Low_Actuals = [];
    Vol_Actuals = [];
    Training_Windows = [];
    
    # For each element of the time series
    while i < len(Time_Series)-Window_Size:
        
        # Take elements from the time series and concatinate them
        Window = Time_Series[i:i+Window_Size];
        
        # Get the response value
        Close_Actuals.append(Extract_Close(Window,Window_Size));
        High_Actuals.append(Extract_High(Window,Window_Size));
        Low_Actuals.append(Extract_Low(Window,Window_Size));
        Vol_Actuals.append(Extract_Vol(Window,Window_Size));
        Training_Windows.append(Make_Window(Window,Window_Size))  
        
        i = i +1
    # Take last window for predictions
    Test_Window = [Make_Window(Time_Series[len(Time_Series)-Window_Size: len(Time_Series)],Window_Size)]
    
    return [Training_Windows,Close_Actuals,High_Actuals,Low_Actuals,Vol_Actuals,Test_Window]

## HTTP Server

In [21]:
def Get_Request_Vals(url):
    x = url.split('=')
    
    Symbol = x[1].split('&')[0]
    Interval = x[2].split('&')[0]
    StartTime= x[3].split('&')[0]
    EndTime= x[4].split('&')[0]
    Limit= x[5].split('&')[0]
    
    return (Symbol,Interval,StartTime,EndTime,Limit)

In [22]:
# Get response from server
def Get_Request(symbol,interval,startTime,endTime,limit):
    # Create request String
    Request = ('http://localhost:8000/?Symbol='+symbol+'&INTERVAL='+interval+'&STARTTIME='+startTime+'&ENDTIME='
    +endTime+'&LIMIT='+limit)
    # Make the request
    Response = Requests.get(Request)
    # Convert JSON to Array & return
    return JSON.loads(Response.json())

In [23]:
class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):

    def do_GET(self):
        self.send_response(200)
        self.end_headers()
        
       # print(self.path)
        
        # Get Request Parameters
        Symbol,Interval,StartTime,EndTime,Limit = Get_Request_Vals(self.path);
        # Request Data From Messaging Layer & Add Candlestick types to time series elements
        Data = Candlestick_Recognition(Get_Request(Symbol,Interval,StartTime,EndTime,Limit));
        # Reverse order so that the oldest data is at the start of the list
        Data = Data[::-1];
        
        # Get start time of next Candlestick for AR Model
        Last_Start_Time = Data[len(Data)-1][0];
        Interval_Diff =  Data[len(Data)-1][0] - Data[len(Data)-2][0] ;
        Next_Start_Time = Last_Start_Time + Interval_Diff;
        Last_Close_Price =  Data[len(Data)-1][3];
        
        
        # Create Window Data sets. Test_Window is the last entry of the time series
        [Training_Windows,Close_Actuals,High_Actuals,Low_Actuals,Vol_Actuals,Test_Window] = Create_Windows(Data,5);
        
        # Build linear auto regressive model
        LinearAR = LinearAutoRegressive(Interval_Diff,Last_Close_Price,Next_Start_Time,Training_Windows,
                                       Close_Actuals,High_Actuals,Low_Actuals,Vol_Actuals,Test_Window,Symbol)
        # predict next 10 intervals
        Predictions = LinearAR.Predict(Limit=100);
        
        
        # remove volume and candlestick
        for i in range(len(Predictions)):
            Predictions[i] = Predictions[i][:5]
        
        Predictions = str(Predictions)
        
        
       # self.response_body('vinvdivdn')
       # return str(Predictions)
      #  self.send_response(b'Hello')
        self.wfile.write(Predictions.encode('utf-8'))
        


In [24]:

Handler = SimpleHTTPRequestHandler

with socketserver.TCPServer(("", PORT), Handler) as httpd:
    print("serving at port", PORT)
    httpd.serve_forever()

serving at port 8005


127.0.0.1 - - [21/May/2019 16:39:26] "GET /?Symbol=ETHUSDT&INTERVAL=1h&STARTTIME=NOW&ENDTIME=LATER&LIMIT=5000000 HTTP/1.1" 200 -


KeyboardInterrupt: 